In [ ]:
from dynamixel_sdk import *  # Dynamixel SDK import
import math
import time

# Initialize PortHandler instance
portHandler = PortHandler('/dev/ttyUSB0')

# Initialize PacketHandler instance
packetHandler = PacketHandler(2.0)

# Initialize GroupSyncWrite instance
groupSyncWrite = GroupSyncWrite(portHandler, packetHandler, 116, 4)  # Address 116 is for goal position, and data size is 4

# Dynamixel motor IDs
HIP_L = 1
KNEE_L = 3
HIP_R = 2
KNEE_R = 4

# Constants
l1 = 7  # Length of the first limb
l2 = 16  # Length of the second limb
stepHeight = 5
stepClearance = 1

def deg_to_dxl(deg):
    return int((deg + 150.0) * (1023 / 300.0))  # Convert from degrees to Dynamixel units. Adjust this function as needed.

def updateServoPos(target1, target2, char_leg):
    if char_leg == 'l':
        hip_goal = deg_to_dxl(target1)
        knee_goal = deg_to_dxl(target2)
        groupSyncWrite.addParam(HIP_L, [DXL_LOBYTE(DXL_LOWORD(hip_goal)), DXL_HIBYTE(DXL_LOWORD(hip_goal)), DXL_LOBYTE(DXL_HIWORD(hip_goal)), DXL_HIBYTE(DXL_HIWORD(hip_goal))])
        groupSyncWrite.addParam(KNEE_L, [DXL_LOBYTE(DXL_LOWORD(knee_goal)), DXL_HIBYTE(DXL_LOWORD(knee_goal)), DXL_LOBYTE(DXL_HIWORD(knee_goal)), DXL_HIBYTE(DXL_HIWORD(knee_goal))])
    elif char_leg == 'r':
        hip_goal = deg_to_dxl(target1)
        knee_goal = deg_to_dxl(target2)
        groupSyncWrite.addParam(HIP_R, [DXL_LOBYTE(DXL_LOWORD(hip_goal)), DXL_HIBYTE(DXL_LOWORD(hip_goal)), DXL_LOBYTE(DXL_HIWORD(hip_goal)), DXL_HIBYTE(DXL_HIWORD(hip_goal))])
        groupSyncWrite.addParam(KNEE_R, [DXL_LOBYTE(DXL_LOWORD(knee_goal)), DXL_HIBYTE(DXL_LOWORD(knee_goal)), DXL_LOBYTE(DXL_HIWORD(knee_goal)), DXL_HIBYTE(DXL_HIWORD(knee_goal))])
    
    groupSyncWrite.txPacket()
    groupSyncWrite.clearParam()

def pos(x, y, leg):
    hipRad = math.atan(x/z)
    hipDeg = math.degrees(hipRad)
    z2 = z / math.cos(hipRad)
    hipRad1 = math.acos((l1 ** 2 + z2 ** 2 - l2 ** 2) / (2 * l1 * z2))
    hipDeg1 = math.degrees(hipRad1)
    kneeRad = math.pi - math.acos((l1 ** 2 + l2 ** 2 - z2 ** 2) / (2 * l1 * l2))
    kneeDeg = math.degrees(kneeRad)
    updateServoPos(hipDeg + hipDeg1, kneeDeg, leg)
    
    

def takeStep(stepLength, stepVelocity):
    for i in range(int(stepLength * 2), int(-stepLength * 2), -1):
        i /= 2.0
        pos(i, stepHeight, 'r')
        pos(-i, stepHeight - stepClearance, 'l')
        time.sleep(stepVelocity / 1000.0)

    for i in range(int(stepLength * 2), int(-stepLength * 2), -1):
        i /= 2.0
        pos(-i, stepHeight - stepClearance, 'r')
        pos(i, stepHeight, 'l')
        time.sleep(stepVelocity / 1000.0)

def initialize():
    for i in range(1070, int(stepHeight * 10), -1):
        i /= 100.0
        pos(0, i, 'l')
        pos(0, i, 'r')

if __name__ == '__main__':
    # Open port
    if portHandler.openPort():
        print("Succeeded to open the port")
    else:
        print("Failed to open the port")
        exit()

    # Set port baudrate
    if portHandler.setBaudRate(57600):
        print("Succeeded to change the baudrate")
    else:
        print("Failed to change the baudrate")
        exit()

    # Enable Dynamixel Torque
    packetHandler.write1ByteTxRx(portHandler, HIP_R, 64, 1)
    packetHandler.write1ByteTxRx(portHandler, KNEE_R, 64, 1)

    # Initialize and start moving
    initialize()
    while True:
        takeStep(2, 0)
